In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import *
import seaborn as sns
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
import mglearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.feature_selection import SelectPercentile
import os
import glob

t=np.zeros((6,3))
t[0:6, 1]=[1,2,3,4,5,6]
print(t)

[[0. 1. 0.]
 [0. 2. 0.]
 [0. 3. 0.]
 [0. 4. 0.]
 [0. 5. 0.]
 [0. 6. 0.]]


In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# 上場企業４２３社の２０年分を学習
# 終値前日比1.03%以上であるかを２クラス分類
# データ比１１：１を１：１に調整（減らした）
# StandardScaler accuracy
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import *
import seaborn as sns
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
import mglearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.feature_selection import SelectPercentile
import os
import glob

# 実行上問題ない注意は非表示にする
warnings.filterwarnings('ignore') 


# In[2]:


# data/kabu1フォルダ内にあるcsvファイルの一覧を取得
files = glob.glob("./*.csv")
print(files)

# In[3]:


# 説明変数となる行列X, 被説明変数となるy2を作成
base = 100 
day_ago = 3
num_sihyou = 8
reset =True
# すべてのCSVファイルから得微量作成
for file in files:
    temp = pd.read_csv(file, header=0, encoding='cp932')
    temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
    temp= temp.iloc[::-1]#上下反対に
    temp2 = np.array(temp)
    print(temp2)
    # 前日比を出すためにbase日後からのデータを取得
    temp3 = np.zeros((len(temp2)-base, num_sihyou))
    temp3[0:len(temp3), 0] = temp2[base:len(temp2), 4] / temp2[base-1:len(temp2)-1, 4]
    temp3[0:len(temp3), 1] = temp2[base:len(temp2), 1] / temp2[base:len(temp2), 4]
    temp3[0:len(temp3), 2] = temp2[base:len(temp2), 2] / temp2[base:len(temp2), 4]
    temp3[0:len(temp3), 3] = temp2[base:len(temp2), 3] / temp2[base:len(temp2), 4]
    temp3[0:len(temp3), 4] = temp2[base:len(temp2), 5].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 5] = temp2[base:len(temp2), 6].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 6] = temp2[base:len(temp2), 7].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 7] = temp2[base:len(temp2), 8].astype(np.float) / temp2[base-1:len(temp2)-1, 8].astype(np.float)
    
    # tempX : 現在の企業のデータ
    tempX = np.zeros((len(temp3), day_ago*num_sihyou))
    
    # 日にちごとに横向きに（day_ago）分並べる
    # sckit-learnは過去の情報を学習できないので、複数日（day_ago）分を特微量に加える必要がある
    # 注：tempX[0:day_ago]分は欠如データが生まれる
    for s in range(0, num_sihyou): 
        for i in range(0, day_ago):
            tempX[i:len(temp3), day_ago*s+i] = temp3[0:len(temp3)-i,s]
             
    # Xに追加
    # X : すべての企業のデータ
    # tempX[0:day_ago]分は削除
    if reset:
        X = tempX[day_ago:]
        reset = False
    else:
        X = np.concatenate((X, tempX[day_ago:]), axis=0)

# 何日後を値段の差を予測するのか
pre_day = 1
# y : pre_day後の終値/当日終値
y = np.zeros(len(X))
y[0:len(y)-pre_day] = X[pre_day:len(X),0]
X = X[:-pre_day]
y = y[:-pre_day]

up_rate =1.03

# データを一旦分別
X_0 = X[y<=up_rate]
X_1 = X[y>up_rate]
y_0 = y[y<=up_rate]
y_1 = y[y>up_rate]
print("y_1:",y_1)

# X_0をX_1とほぼ同じ数にする
X_drop, X_t, y_drop, y_t = train_test_split(X_0, y_0, test_size=0.09, random_state=0)

# 分別したデータの結合
X_ = np.concatenate((X_1, X_t), axis=0)
y_ = np.concatenate((y_1, y_t))


# In[4]:


# 確認
# 何もしないときの比率
print("X.shape: ", X.shape)

print("yの割合")
# yc：翌日の終値/当日の終値がup_rateより上か
yc = np.zeros(len(y))
for i in range(0, len(yc)):
    if y[i] <= up_rate:
        yc[i] = 0
    else:
        yc[i] = 1

pd_yc = pd.DataFrame(yc)
print(pd_yc[0].value_counts())

# 確認
# X_0をX_1の数を調整後の比率
print("X_.shape: ", X_.shape)

print("y_の割合")
# yc：翌日の終値/当日の終値がup_rateより上か
yc_ = np.zeros(len(y_))
for i in range(0, len(yc_)):
    if y_[i] <= up_rate:
        yc_[i] = 0
    else:
        yc_[i] = 1

pd_yc_ = pd.DataFrame(yc_)
print(pd_yc_[0].value_counts())

X_train, X_test, y_train, y_test = train_test_split(X_, y_, random_state=0)
print("X_train:", X_train)
# y_train_,y_test2：翌日の終値/当日の終値がup_rateより上か
y_train2 = np.zeros(len(y_train))
for i in range(0, len(y_train2)):
    if y_train[i] <= up_rate:
        y_train2[i] = 0
    else:
        y_train2[i] = 1
        
y_test2 = np.zeros(len(y_test))
for i in range(0, len(y_test2)):
    if y_test[i] <= up_rate:
        y_test2[i] = 0
    else:
        y_test2[i] = 1

print("start time: ", datetime.now().strftime("%H:%M:%S"))

pipe = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(max_iter=200000, alpha=0.001))])
param_grid = {'classifier__hidden_layer_sizes': [(10,), (100,), (500,)]}
mlp=MLPClassifier(max_iter=200000, alpha=0.001,random_state=0)
mlp.fit(X_train,y_train)
print("accuracy on test set:{:.3f}".format(mlp.score(X_test,y_test)))
grid = GridSearchCV(pipe, param_grid=param_grid, n_jobs=1, cv=2 ,return_train_score=False, scoring="accuracy")
grid.fit(X_train, y_train)

print(grid.cv_results_['mean_test_score'])
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("over time: ", datetime.now().strftime("%H:%M:%S"))

conf = confusion_matrix(y_test, grid.predict(X_test))
print(conf)


# 一目均衡表,転換線,基準線,先行スパン1,先行スパン2,25日ボリンジャーバンド, 曜日 追加
# 説明変数となる行列X, 被説明変数となるy2を作成
base = 100 
day_ago = 3
num_sihyou = 16
reset =True
# すべてのCSVファイルから得微量作成
for file in files:
    temp = pd.read_csv(file, header=0, encoding='cp932')
    temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
    temp= temp.iloc[::-1]#上下反対に
    temp2 = np.array(temp)
    
    # 前日比を出すためにbase日後からのデータを取得
    temp3 = np.zeros((len(temp2)-base, num_sihyou))
    temp3[0:len(temp3), 0] = temp2[base:len(temp2), 4] / temp2[base-1:len(temp2)-1, 4]
    temp3[0:len(temp3), 1] = temp2[base:len(temp2), 1] / temp2[base:len(temp2), 4]
    temp3[0:len(temp3), 2] = temp2[base:len(temp2), 2] / temp2[base:len(temp2), 4]
    temp3[0:len(temp3), 3] = temp2[base:len(temp2), 3] / temp2[base:len(temp2), 4]
    temp3[0:len(temp3), 4] = temp2[base:len(temp2), 5].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 5] = temp2[base:len(temp2), 6].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 6] = temp2[base:len(temp2), 7].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 7] = temp2[base:len(temp2), 8].astype(np.float) / temp2[base-1:len(temp2)-1, 8].astype(np.float)
    
     # 一目均衡表を追加します (9,26, 52) 
    para1 =9
    para2 = 26
    para3 = 52
    temp2_2 = np.c_[temp2, np.zeros((len(temp2), 3))]
    p1 = 9
    p2 = 10
    p3 =11
    
    # 転換線 = （過去(para1)日間の高値 + 安値） ÷ 2
    for i in range(para1, len(temp2)):
        tmp_high =temp2[i-para1+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para1+1:i+1,3].astype(np.float)
        temp2_2[i, p1] = (np.max(tmp_high) + np.min(tmp_low)) / 2 /temp2[i, 4]
        
    temp3[0:len(temp3), 8] = temp2_2[base:len(temp2), p1]

    # 基準線 = （過去(para2)日間の高値 + 安値） ÷ 2
    for i in range(para2, len(temp2)):
        tmp_high =temp2[i-para2+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para2+1:i+1,3].astype(np.float)
        temp2_2[i, p2] = (np.max(tmp_high) + np.min(tmp_low)) / 2 /temp2[i, 4]
    temp3[0:len(temp3), 9] = temp2_2[base:len(temp2), p2]
        

    # 先行スパン1 = ｛ （転換値+基準値） ÷ 2 ｝を(para2)日先にずらしたもの
    temp3[0:len(temp3), 10] = (temp2_2[base-para2:len(temp2)-para2, p1] + temp2_2[base-para2:len(temp2)-para2, p2]) /2 /temp2[base:len(temp2), 4]

    # 先行スパン2 = ｛ （過去(para3)日間の高値+安値） ÷ 2 ｝を(para2)日先にずらしたもの
    for i in range(para3, len(temp2)):
        tmp_high =temp2[i-para3+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para3+1:i+1,3].astype(np.float)
        temp2_2[i, p3] = (np.max(tmp_high) + np.min(tmp_low)) / 2 /temp2[i, 4]
    temp3[0:len(temp3), 11] = temp2_2[base-para2:len(temp2)-para2, p3]

    # 25日ボリンジャーバンド（±1, 2シグマ）を追加します
    parab = 25
    for i in range(base, len(temp2)):
        tmp25 = temp2[i-parab+1:i+1,4].astype(np.float)
        temp3[i-base,12] = np.mean(tmp25) + 1.0* np.std(tmp25) 
        temp3[i-base,13] = np.mean(tmp25) - 1.0* np.std(tmp25) 
        temp3[i-base,14] = np.mean(tmp25) + 2.0* np.std(tmp25) 
        temp3[i-base,15] = np.mean(tmp25) - 2.0* np.std(tmp25)
        
    # tempX : 現在の企業のデータ
    tempX = np.zeros((len(temp3), day_ago*num_sihyou))
    
    # 日にちごとに横向きに（day_ago）分並べる
    # sckit-learnは過去の情報を学習できないので、複数日（day_ago）分を特微量に加える必要がある
    # 注：tempX[0:day_ago]分は欠如データが生まれる
    for s in range(0, num_sihyou): 
        for i in range(0, day_ago):
            tempX[i:len(temp3), day_ago*s+i] = temp3[0:len(temp3)-i,s]
             
    # 曜日情報の追加
    ddata = pd.to_datetime(temp['日付'], format='%Y%m%d')
    daydata = ddata[base:len(temp2)].dt.dayofweek
    daydata_dummies = pd.get_dummies(daydata, columns=['Yobi'])
    daydata2 = np.array(daydata_dummies)
    tempX = np.concatenate((tempX, daydata2), axis=1)
    
    # Xに追加
    # X : すべての企業のデータ
    # tempX[0:day_ago]分は削除
    if reset:
        X = tempX[day_ago:]
        reset = False
    else:
        X = np.concatenate((X, tempX[day_ago:]), axis=0)

# 何日後を値段の差を予測するのか
pre_day = 1
# y : pre_day後の終値/当日終値
y = np.zeros(len(X))
y[0:len(y)-pre_day] = X[pre_day:len(X),0]
X = X[:-pre_day]
y = y[:-pre_day]

up_rate =1.03
# y2：翌日の終値/当日の終値がup_rateより上か
y2 = np.zeros(len(y))
for i in range(0, len(y2)):
    if y[i] <= up_rate:
        y2[i] = 0
    else:
        y2[i] = 1

# データを一旦分別
X_0 = X[y2==0]
X_1 = X[y2==1]
y_0 = y2[y2==0]
y_1 = y2[y2==1]

# X_0をX_1とほぼ同じ数にする
X_dummy, X_t, y_dummy, y_t = train_test_split(X_0, y_0, test_size=0.09, random_state=0)

# 分別したデータの結合
X_ = np.concatenate((X_1, X_t), axis=0)
y_ = np.concatenate((y_1, y_t))


# In[ ]:


print("start time: ", datetime.now().strftime("%H:%M:%S"))

X_train, X_test, y_train, y_test = train_test_split(X_, y_, random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(max_iter=200000, alpha=0.001))])
param_grid = {'classifier__hidden_layer_sizes': [(10,), (100,), (500,)]}

mlp=MLPClassifier(max_iter=200000, alpha=0.001,random_state=0)
mlp.fit(X_train,y_train)

grid = GridSearchCV(pipe, param_grid=param_grid, n_jobs=1, cv=2 ,return_train_score=False, scoring="accuracy")
grid.fit(X_train, y_train)

print(grid.cv_results_['mean_test_score'])
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("over time: ", datetime.now().strftime("%H:%M:%S"))

conf = confusion_matrix(y_test, grid.predict(X_test))
print(conf)

['.\\6501TimeChart.csv', '.\\6645TimeChart.csv', '.\\6752TimeChart.csv', '.\\6758TimeChart.csv', '.\\6856TimeChart.csv', '.\\7701TimeChart.csv']
[[20020109 4805.0 4845.0 ... '         --' '         --' 877000]
 [20020110 4820.0 4830.0 ... '         --' '         --' 882600]
 [20020111 4805.0 4815.0 ... '         --' '         --' 1173800]
 ...
 [20191226 4599.0 4621.0 ... '    4350.16' '       4186' 1706000]
 [20191227 4623.0 4640.0 ... '    4370.88' '    4195.16' 1632100]
 [20191230 4606.0 4636.0 ... '     4390.8' '       4204' 2303600]]
[[20020115 279 280 ... '         --' '         --' 727000]
 [20020116 271 272 ... '         --' '         --' 389000]
 [20020117 266 267 ... '         --' '         --' 416000]
 ...
 [20191226 3480 3490 ... '     3363.8' '    3023.76' 264400]
 [20191227 3500 3505 ... '       3375' '     3034.9' 325300]
 [20191230 3495 3495 ... '     3382.6' '    3045.36' 399900]]
[[20020109 1767.0 1795.0 ... '         --' '         --' 10014000]
 [20020110 1790.0 1815

ValueError: Unknown label type: (array([1.00415512, 1.00215517, 1.05172414, ..., 1.03636364, 0.97039474,
       1.00666667]),)

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import *
import seaborn as sns
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
import mglearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.feature_selection import SelectPercentile
import os
import glob
from sklearn.svm import SVC

files = glob.glob("./old*.csv")

# 説明変数となる行列X, 被説明変数となるy2を作成
base = 100 
day_ago = 7
pre_day=1
reset =True

#出来高やボリンジャーバンドなど指標の数
num_sihyou = 16

# すべてのCSVファイルから得微量作成
for file in files:
    temp = pd.read_csv(file, header=0, encoding='cp932')
    temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
    temp= temp.iloc[::-1]#上下反対に
    temp2 = np.array(temp)
    print(file)
    print(len(temp2))
    # 前日比を出すためにbase日後からのデータを取得
    temp3 = np.zeros((len(temp2)-base, num_sihyou))
    temp3[0:len(temp3), 0] = temp2[base:len(temp2), 4].astype(np.float) / temp2[base-1:len(temp2)-1, 4].astype(np.float)
    temp3[0:len(temp3), 1] = temp2[base:len(temp2), 1].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 2] = temp2[base:len(temp2), 2].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 3] = temp2[base:len(temp2), 3].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 4] = temp2[base:len(temp2), 5].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 5] = temp2[base:len(temp2), 6].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 6] = temp2[base:len(temp2), 7].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 7] = temp2[base:len(temp2), 8].astype(np.float) / temp2[base-1:len(temp2)-1, 8].astype(np.float)
    
     # 一目均衡表を追加 (9,26, 52) 
    para1 =9
    para2 = 26
    para3 = 52
    temp2_2 = np.c_[temp2, np.zeros((len(temp2), 3))]
    
    # 転換線 = （過去(para1)日間の高値 + 安値） ÷ 2
    for i in range(para1, len(temp2)):
        tmp_high =temp2[i-para1+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para1+1:i+1,3].astype(np.float)
        temp2_2[i, 9] = (np.max(tmp_high) + np.min(tmp_low))/2
        
    temp3[0:len(temp3), 8] = temp2_2[base:len(temp2), 9]/temp2[base:len(temp2), 4]

    # 基準線 = （過去(para2)日間の高値 + 安値） ÷ 2
    for i in range(para2, len(temp2)):
        tmp_high =temp2[i-para2+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para2+1:i+1,3].astype(np.float)
        temp2_2[i, 10] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 9] = temp2_2[base:len(temp2), 10]/temp2[base:len(temp2), 4]
        
    # 先行スパン1 = ｛ （転換値+基準値） ÷ 2 ｝を(para2)日先にずらしたもの
    temp3[0:len(temp3), 10] = (temp2_2[base-para2:len(temp2)-para2, 9] + temp2_2[base-para2:len(temp2)-para2, 10]) /2 /temp2[base:len(temp2), 4]

    # 先行スパン2 = ｛ （過去(para3)日間の高値+安値） ÷ 2 ｝を(para2)日先にずらしたもの
    for i in range(para3, len(temp2)):
        tmp_high =temp2[i-para3+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para3+1:i+1,3].astype(np.float)
        temp2_2[i, 11] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 11] = temp2_2[base-para2:len(temp2)-para2, 11]/temp2[base:len(temp2), 4]

    # 25日ボリンジャーバンド（±1, 2シグマ）を追加
    parab = 25
    for i in range(base, len(temp2)):
        tmp25 = temp2[i-parab+1:i+1,4].astype(np.float)
        temp3[i-base,12] = np.mean(tmp25) + 1.0* np.std(tmp25) 
        temp3[i-base,13] = np.mean(tmp25) - 1.0* np.std(tmp25) 
        temp3[i-base,14] = np.mean(tmp25) + 2.0* np.std(tmp25) 
        temp3[i-base,15] = np.mean(tmp25) - 2.0* np.std(tmp25)
     
    # tempX : 現在の企業のデータ
    tempX = np.zeros((len(temp3), day_ago*num_sihyou))
    
    # 日にちごとに横向きに（day_ago）分並べる
    for s in range(0, num_sihyou): 
        for i in range(0, day_ago):
            tempX[i:len(temp3), day_ago*s+i] = temp3[0:len(temp3)-i,s]
             
    # Xに追加
    # X : すべての企業のデータ
    if reset:
        X = tempX[day_ago:]
        reset = False
    else:
        X = np.concatenate((X, tempX[day_ago:]), axis=0)

# y : pre_day後の終値/当日終値
y = np.zeros(len(X))
y[0:len(y)-pre_day] = X[pre_day:len(X),0]

up_rate =1.02
down_rate=0.98

print("X.shape: ", X.shape)
print("yの割合")

#それぞれの範囲でデータ数をカウント
a=b=c=d=0
yc = np.zeros(len(y))
for i in range(0, len(yc)):
    if y[i] <= down_rate:
        yc[i] = 0
        a+=1
    elif y[i]>down_rate and y[i]<=1:
        yc[i]=1
        b+=1
    elif y[i] <=up_rate and y[i]>1:
        yc[i] = 2
        c+=1
    else:
        yc[i] = 3
        d+=1
print("0",a)
print("1",b)
print("2",c)
print("3",d)

X_train, X_test, y_train, y_test = train_test_split(X, yc, random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(max_iter=200000, alpha=0.001))])
param_grid = {'classifier__hidden_layer_sizes': [(10,), (100,), (500,)]}

grid = GridSearchCV(pipe, param_grid=param_grid, n_jobs=1, cv=2 ,return_train_score=False, scoring="accuracy")
grid.fit(X_train, y_train)

print(grid.cv_results_['mean_test_score'])
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
prediction_test=grid.predict(X_test)
print(prediction_test)
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))


reset=True
#全く違う会社、違う時期のデータで予測
files = glob.glob("./recent*.csv")
for file in files:
    temp = pd.read_csv(file, header=0, encoding='cp932')
    temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
    temp= temp.iloc[::-1]#上下反対に
    temp2 = np.array(temp)
    print(file)
    print(len(temp2))
    # 前日比を出すためにbase日後からのデータを取得
    temp3 = np.zeros((len(temp2)-base, num_sihyou))
    temp3[0:len(temp3), 0] = temp2[base:len(temp2), 4].astype(np.float) / temp2[base-1:len(temp2)-1, 4].astype(np.float)
    temp3[0:len(temp3), 1] = temp2[base:len(temp2), 1].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 2] = temp2[base:len(temp2), 2].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 3] = temp2[base:len(temp2), 3].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 4] = temp2[base:len(temp2), 5].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 5] = temp2[base:len(temp2), 6].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 6] = temp2[base:len(temp2), 7].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 7] = temp2[base:len(temp2), 8].astype(np.float) / temp2[base-1:len(temp2)-1, 8].astype(np.float)
    
     # 一目均衡表を追加 (9,26, 52) 
    para1 =9
    para2 = 26
    para3 = 52
    temp2_2 = np.c_[temp2, np.zeros((len(temp2), 3))]
    
    # 転換線 = （過去(para1)日間の高値 + 安値） ÷ 2
    for i in range(para1, len(temp2)):
        tmp_high =temp2[i-para1+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para1+1:i+1,3].astype(np.float)
        temp2_2[i, 9] = (np.max(tmp_high) + np.min(tmp_low))/2
        
    temp3[0:len(temp3), 8] = temp2_2[base:len(temp2), 9]/temp2[base:len(temp2), 4]

    # 基準線 = （過去(para2)日間の高値 + 安値） ÷ 2
    for i in range(para2, len(temp2)):
        tmp_high =temp2[i-para2+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para2+1:i+1,3].astype(np.float)
        temp2_2[i, 10] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 9] = temp2_2[base:len(temp2), 10]/temp2[base:len(temp2), 4]
        
    # 先行スパン1 = ｛ （転換値+基準値） ÷ 2 ｝を(para2)日先にずらしたもの
    temp3[0:len(temp3), 10] = (temp2_2[base-para2:len(temp2)-para2, 9] + temp2_2[base-para2:len(temp2)-para2, 10]) /2 /temp2[base:len(temp2), 4]

    # 先行スパン2 = ｛ （過去(para3)日間の高値+安値） ÷ 2 ｝を(para2)日先にずらしたもの
    for i in range(para3, len(temp2)):
        tmp_high =temp2[i-para3+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para3+1:i+1,3].astype(np.float)
        temp2_2[i, 11] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 11] = temp2_2[base-para2:len(temp2)-para2, 11]/temp2[base:len(temp2), 4]

    # 25日ボリンジャーバンド（±1, 2シグマ）を追加
    parab = 25
    for i in range(base, len(temp2)):
        tmp25 = temp2[i-parab+1:i+1,4].astype(np.float)
        temp3[i-base,12] = np.mean(tmp25) + 1.0* np.std(tmp25) 
        temp3[i-base,13] = np.mean(tmp25) - 1.0* np.std(tmp25) 
        temp3[i-base,14] = np.mean(tmp25) + 2.0* np.std(tmp25) 
        temp3[i-base,15] = np.mean(tmp25) - 2.0* np.std(tmp25)
     
    # tempX : 現在の企業のデータ
    tempX = np.zeros((len(temp3), day_ago*num_sihyou))
    
    # 日にちごとに横向きに（day_ago）分並べる
    for s in range(0, num_sihyou): 
        for i in range(0, day_ago):
            tempX[i:len(temp3), day_ago*s+i] = temp3[0:len(temp3)-i,s]
    print()
    # Xに追加
    # X : すべての企業のデータ
    
    if reset:
        X_recent = tempX[len(temp3)-1,:]
        reset = False
    else:
        X_recent = np.vstack((X_recent, tempX[len(temp3)-1,:]))
    print("X_recent",X_recent.shape)
print("X_predict",X_recent.shape)
prediction=grid.predict(X_recent)
print(prediction)
print(len(prediction))

.\old2587TimeChart.csv
1589
.\old4755TimeChart.csv
4395
.\old6022TimeChart.csv
4048
.\old6103TimeChart.csv
4383
.\old6111TimeChart.csv
3485
.\old6113TimeChart.csv
4383
.\old6118TimeChart.csv
4383
.\old6138TimeChart.csv
4375
.\old6143TimeChart.csv
4383
.\old6236TimeChart.csv
917
.\old6255TimeChart.csv
3061
.\old6267TimeChart.csv
2717
.\old6282TimeChart.csv
4383
.\old6294TimeChart.csv
3993
.\old6301TimeChart.csv
4383
.\old6328TimeChart.csv
4374
.\old6342TimeChart.csv
3900
.\old6345TimeChart.csv
4383
.\old6362TimeChart.csv
4383
.\old6367TimeChart.csv
4383
.\old6373TimeChart.csv
4383
.\old6383TimeChart.csv
4383
.\old6408TimeChart.csv
3586
.\old6428TimeChart.csv
4382
.\old6430TimeChart.csv
4204
.\old6472TimeChart.csv
4383
.\old6486TimeChart.csv
4380
.\old6494TimeChart.csv
4374
.\old6501TimeChart.csv
4409
.\old6645TimeChart.csv
4406
.\old6752TimeChart.csv
4409
.\old6758TimeChart.csv
4409
.\old6856TimeChart.csv
4406
.\old6861TimeChart.csv
4395
.\old7013TimeChart.csv
4382
.\old7201TimeChart.cs

In [42]:
files = glob.glob("./recent6297.csv")
temp = pd.read_csv(file, header=0, encoding='cp932')
temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
#print(temp)
temp2 = np.array(temp)
print(temp2)
temp= temp.iloc[::-1]#上下反対に
print("-----------------------")

temp2_new = np.array(temp)
print(temp2_new)

[[20220214 519 525 ... '     521.56' '      542.6' 1600]
 [20220210 527 529 ... '     522.32' '     543.73' 1600]
 [20220209 520 529 ... '     522.96' '     544.62' 1100]
 ...
 [20181001 854 854 ... '         --' '         --' 32400]
 [20180928 704 704 ... '         --' '         --' 83900]
 [20180927 696 699 ... '         --' '         --' 2957100]]
-----------------------
[[20180927 696 699 ... '         --' '         --' 2957100]
 [20180928 704 704 ... '         --' '         --' 83900]
 [20181001 854 854 ... '         --' '         --' 32400]
 ...
 [20220209 520 529 ... '     522.96' '     544.62' 1100]
 [20220210 527 529 ... '     522.32' '     543.73' 1600]
 [20220214 519 525 ... '     521.56' '      542.6' 1600]]
